# Converting a pdf into DNA code words

In [30]:
import numpy as np
# from asyncore import read
import PyPDF2 
data = open("/Users/gnbhavithran/Desktop/AEJM-S-22-00711.pdf", 'rb')
f = open("/Users/gnbhavithran/Desktop/Indel_error1s.pdf", 'w')
data_pdf = PyPDF2.PdfFileReader(data)

In [8]:
import vtcode as vt
import Kernel_code as kc

In [9]:
n = 14
binary_14_digit = vt.VTCode(n, 2, correct_substitutions=True)

In [10]:
length = binary_14_digit.k

In [52]:
data_pdf_bin =[''] * data_pdf.numPages
DNA_encoded_pdf = [''] * data_pdf.numPages
for idx, i in enumerate(data_pdf.pages):
    what_it_is = i.extract_text().encode('UTF-8')
    data_pdf_bin[idx] = []
    DNA_encoded_pdf[idx] = []
    for j in what_it_is:
        val = list(np.binary_repr(j, length))
        val = [int(x) for x in val]
        data_pdf_bin[idx].append(val)
        encode_word = binary_14_digit.encode(val)
        DNA_encoded_pdf[idx].append(encode_word)

In [69]:
sum = 0
pdf_rebulding = [''] * data_pdf.numPages

for idx, i in enumerate(DNA_encoded_pdf):
    pdf_rebulding[idx] = []
    for things in DNA_encoded_pdf[idx]:
        decode_val = binary_14_digit.decode(things)
        decode_val_decimal = int("".join(str(i) for i in decode_val),2)
        pdf_rebulding[idx].append(decode_val_decimal)

In [82]:
for i in range(data_pdf.numPages):
    comparison_value_orginal = data_pdf.pages[i].extract_text().encode('UTF-8')
    for j in range(len(pdf_rebulding[i])):
        if pdf_rebulding[i][j] != comparison_value_orginal[j]:
            print(i)
            print(pdf_rebulding[i][j])
            print(comparison_value_orginal[j])
            print("Idiot!")
            break

In [91]:
pdf_str_rebulding =[''] * len(pdf_rebulding)
for i in range(len(pdf_rebulding)):
    pdf_str_rebulding[i] = []
    for j in range(len(pdf_rebulding[i])):
        pdf_str_rebulding[i].append(chr(pdf_rebulding[i][j]))

In [163]:
type(pdf_rebulding[0])

list

In [167]:
pdf_str_rebulding[1]

['O',
 'c',
 't',
 'o',
 'b',
 'e',
 'r',
 ' ',
 '1',
 '1',
 ',',
 ' ',
 '2',
 '0',
 '2',
 '2',
 ' ',
 '1',
 '3',
 ':',
 '2',
 '8',
 ' ',
 'W',
 'S',
 'P',
 'C',
 '/',
 'I',
 'N',
 'S',
 'T',
 'R',
 'U',
 'C',
 'T',
 'I',
 'O',
 'N',
 ' ',
 'F',
 'I',
 'L',
 'E',
 ' ',
 'o',
 'u',
 't',
 'p',
 'u',
 't',
 '\n',
 'A',
 's',
 'i',
 'a',
 'n',
 '-',
 'E',
 'u',
 'r',
 'o',
 'p',
 'e',
 'a',
 'n',
 ' ',
 'J',
 'o',
 'u',
 'r',
 'n',
 'a',
 'l',
 ' ',
 'o',
 'f',
 ' ',
 'M',
 'a',
 't',
 'h',
 'e',
 'm',
 'a',
 't',
 'i',
 'c',
 's',
 '\n',
 'V',
 'o',
 'l',
 '.',
 ' ',
 '1',
 ',',
 ' ',
 'N',
 'o',
 '.',
 ' ',
 '1',
 ' ',
 '(',
 '2',
 '0',
 '1',
 '0',
 ')',
 ' ',
 '1',
 'â',
 '\x80',
 '\x93',
 '1',
 '2',
 '\n',
 'Â',
 '©',
 'W',
 'o',
 'r',
 'l',
 'd',
 ' ',
 'S',
 'c',
 'i',
 'e',
 'n',
 't',
 'i',
 'f',
 'i',
 'c',
 ' ',
 'P',
 'u',
 'b',
 'l',
 'i',
 's',
 'h',
 'i',
 'n',
 'g',
 ' ',
 'C',
 'o',
 'm',
 'p',
 'a',
 'n',
 'y',
 '\n',
 'D',
 'O',
 'I',
 ':',
 ' ',
 '1',
 '0',
 '.',
 '1',


In [93]:
data_pdf.pages[1].extract_text()[1]

'c'

In [150]:
file=open("/Users/gnbhavithran/Python_github/My_storage_to_remind/VT_codes/hello.pdf","wb")


In [151]:
k = []
for what in data_pdf.pages:
    k.append(what.extract_text().encode('UTF-8'))

In [168]:
C = [''] * len(pdf_rebulding)
for coun, values in enumerate(pdf_rebulding):
    C[coun] = ''.join(chr(i) for i in values)

In [170]:
C[1]

'October 11, 2022 13:28 WSPC/INSTRUCTION FILE output\nAsian-European Journal of Mathematics\nVol. 1, No. 1 (2010) 1â\x80\x9312\nÂ©World Scientific Publishing Company\nDOI: 10.1142/DOI Number\nIndel Error Correction Codes for DNA Digital Data Storage and\nRetrieval\nNallappaBhavithran G\nDepartment of Mathematics, Vellore Institute of Technology,\nVellore, Tamil Nadu, 632014, India.\nnallabhavitran@gmail.com\nSelvakumar Râ\x88\x97\nDepartment of Mathematics, Vellore Institute of Technology,\nVellore, Tamil Nadu, 632014, India.\nrselvakuamr@vit.ac.in\nReceived (Day Month Year)\nRevised (Day Month Year)\nA procedure for storage and retrieval of Digital information in DNA strings is dis-\ncussed by constructing an error correction code which is capable of correcting indel\nerrors and maintaining the stability of DNA strings. For correcting indel errors, Var-\nshamovâ\x80\x93Tenengolts algorithm is used. In the VT algorithm, syndrome is calculated\nusing the position values of the retrieved

In [1]:
import numpy

In [1]:
a = '110001100'
val = [int(x) for x in a]

In [2]:
val == [1, 1, 0, 0, 0, 1, 1, 0, 0]

True

In [5]:
b = [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
len(val) == len(b)

False

In [6]:
len(val)

19

In [7]:
len(b)

20

In [8]:
val

[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1]